In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Neural Network Regression with TensorFlow

In [ ]:
import tensorflow as tf

import matplotlib.pyplot as plt


In [ ]:
# Read in the insurance dataset
insurance = pd.read_csv("../input/insurance/insurance.csv")

In [ ]:
insurance

# Preprocessing: normalization and standardization

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder


#Create a column transformer, passing column names to normalize and how to normalize
ct = make_column_transformer(
    (MinMaxScaler(), ["age", "bmi", "children"]),
    (OneHotEncoder(handle_unknown="ignore"), ["sex", "smoker", "region"]) #categorials one-hot encode
)


In [ ]:
#Create X and y values
X = insurance.drop("charges", axis=1) #this is the DV so drop it from training
y = insurance["charges"] #here's the DV

In [ ]:
#Build train and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) #test set is 20%

In [ ]:
len(X), len(X_train), len(X_test) #check sizes

In [ ]:
#Fit the column transformer to the training data ONLY!
ct.fit(X_train)

In [ ]:
#Tranform training and test data with normalization (MinMaxScaler) 
X_train_normal = ct.transform(X_train)
X_test_normal = ct.transform(X_test)

Compare data we had before preprocessing with what we have now!

In [ ]:
X_train.loc[0] #this is what we had to start with

In [ ]:
X_train_normal[0] #this is what we have now, they are all between 0 and 1 with the categorials
#one-hot encoded to numericals

In [ ]:
X_train_normal.shape #more columns due to one-hot encoding of categoricals

In [ ]:
X_train.shape

# Build a model

In [ ]:
tf.random.set_seed(42)

insurance_model_1 = tf.keras.Sequential([
  tf.keras.layers.Dense(100),
  tf.keras.layers.Dense(10),
  tf.keras.layers.Dense(1)  
])

insurance_model_1.compile(loss=tf.keras.losses.mae,
                          optimizer=tf.keras.optimizers.Adam(), #default lr 0.001
                          metrics=["mae"])

In [ ]:
#fit the model

history = insurance_model_1.fit(X_train_normal, y_train, epochs=100)

In [ ]:
#evaluate
insurance_model_1.evaluate(X_test_normal, y_test)

In [ ]:

pd.DataFrame(history.history).plot()


# Try building a new model with non-linearities using RELU

In [ ]:
tf.random.set_seed(42)

insurance_model_2 = tf.keras.Sequential([
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(40, activation='relu'),
  tf.keras.layers.Dense(54, activation='relu'),
  tf.keras.layers.Dense(1)  
])

insurance_model_2.compile(loss=tf.keras.losses.mae,
                          optimizer=tf.keras.optimizers.Adam(),
                          metrics=["mae"])

In [ ]:
#fit the model

history = insurance_model_2.fit(X_train_normal, y_train, epochs=100, verbose=0)

In [ ]:
insurance_model_2.evaluate(X_test_normal, y_test)

In [ ]:
pd.DataFrame(history.history).plot()

# Still seems to be improving at epoch 100! Try 500 epochs for a new model. Or could use a callback to stop when no improving

In [ ]:
tf.random.set_seed(42)

insurance_model_3 = tf.keras.Sequential([
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(40, activation='relu'),
  tf.keras.layers.Dense(54, activation='relu'),
  tf.keras.layers.Dense(1)  
])

insurance_model_3.compile(loss=tf.keras.losses.mae,
                          optimizer=tf.keras.optimizers.Adam(),
                          metrics=["mae"])

In [ ]:
#fit the model

history = insurance_model_3.fit(X_train_normal, y_train, epochs=500,  verbose=0)

In [ ]:
insurance_model_3.evaluate(X_test_normal, y_test)

In [ ]:
pd.DataFrame(history.history).plot()

# Still seems to be improving at epoch 500. Try 2000 epochs for a new model!

In [ ]:

insurance_model_4 = tf.keras.Sequential([
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(40, activation='relu'),
  tf.keras.layers.Dense(54, activation='relu'),
  tf.keras.layers.Dense(1)  
])

insurance_model_4.compile(loss=tf.keras.losses.mae,
                          optimizer=tf.keras.optimizers.Adam(),
                          metrics=["mae"])

In [ ]:
#fit the model

history = insurance_model_4.fit(X_train_normal, y_train, epochs=2000, verbose=0)

In [ ]:
insurance_model_4.evaluate(X_test_normal, y_test)

In [ ]:
pd.DataFrame(history.history).plot()

# So that's good! We have an mean absolute error of 1279 now!

# Let's try a new model, but we increase our learning rate each epoch. Starting at 0.0001

In [ ]:
insurance_model_5 = tf.keras.Sequential([
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(40, activation='relu'),
  tf.keras.layers.Dense(54, activation='relu'),
  tf.keras.layers.Dense(1)  
])

insurance_model_5.compile(loss=tf.keras.losses.mae,
                          optimizer=tf.keras.optimizers.Adam(lr=0.01),
                          metrics=["mae"])

In [ ]:
# Fit the model USE THE LEARNING RATE CALLBACK
# increase 10 to power of epoch/20
lr_schedular = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-4 * 10**(epoch/20))


history5 = insurance_model_5.fit(X_train_normal, y_train, epochs=100, verbose=0, 
                      callbacks=[lr_schedular]) 

In [ ]:
#it's gonna be bad at the end with really big lrs but here it is !
insurance_model_5.evaluate(X_test_normal, y_test)

In [ ]:
#let's see how the lr increases
pd.DataFrame(history5.history)

In [ ]:
#Plot learning rate vs loss
lrs = 1e-4 * (10 ** (tf.range(100)/20))
len(lrs)
lrs

In [ ]:
plt.figure(figsize=(10,7))
#log scaling on x axis
plt.semilogx(lrs, history5.history["loss"])
plt.xlabel("Learning Rate")
plt.ylabel("Loss")
plt.title("Learning Rate vs. Loss")

In [ ]:
1e-2 #is this what to try for lr?

In [ ]:
insurance_model_6 = tf.keras.Sequential([
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(40, activation='relu'),
  tf.keras.layers.Dense(54, activation='relu'),
  tf.keras.layers.Dense(1)  
])

insurance_model_6.compile(loss=tf.keras.losses.mae,
                          optimizer=tf.keras.optimizers.Adam(lr=0.01),
                          metrics=["mae"])

In [ ]:
#fit the model

history6 = insurance_model_6.fit(X_train_normal, y_train, epochs=2000, verbose=0)

In [ ]:
insurance_model_6.evaluate(X_test_normal, y_test)

No that didn't help! What about trying between 1e-2 and 1e-1?

In [ ]:
insurance_model_7 = tf.keras.Sequential([
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(40, activation='relu'),
  tf.keras.layers.Dense(54, activation='relu'),
  tf.keras.layers.Dense(1)  
])

insurance_model_7.compile(loss=tf.keras.losses.mae,
                          optimizer=tf.keras.optimizers.Adam(lr=0.015),
                          metrics=["mae"])

In [ ]:
#fit the model

history6 = insurance_model_7.fit(X_train_normal, y_train, epochs=2000, verbose=0)

In [ ]:
insurance_model_7.evaluate(X_test_normal, y_test)

# So MAE=1453 vs. the best of 1279 in model 4, which was default lr of 0.001

What about a learning rate between 1e-3 and 1e-2? So 0.0015

In [ ]:
insurance_model_8 = tf.keras.Sequential([
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(40, activation='relu'),
  tf.keras.layers.Dense(54, activation='relu'),
  tf.keras.layers.Dense(1)  
])

insurance_model_8.compile(loss=tf.keras.losses.mae,
                          optimizer=tf.keras.optimizers.Adam(lr=0.0015),
                          metrics=["mae"])

In [ ]:
history8 = insurance_model_8.fit(X_train_normal, y_train, epochs=2000, verbose=0)

In [ ]:
insurance_model_8.evaluate(X_test_normal, y_test)

In [ ]:
pd.DataFrame(history8.history).plot()

# Finally I wanted to finish now with just lr=0.001 and 10k epochs!

In [ ]:
insurance_model_9 = tf.keras.Sequential([
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(40, activation='relu'),
  tf.keras.layers.Dense(54, activation='relu'),
  tf.keras.layers.Dense(1)  
])

insurance_model_9.compile(loss=tf.keras.losses.mae,
                          optimizer=tf.keras.optimizers.Adam(0.001),
                          metrics=["mae"])

In [ ]:
history9 = insurance_model_9.fit(X_train_normal, y_train, epochs=10000, verbose=0)

In [ ]:
insurance_model_9.evaluate(X_test_normal, y_test)

# Well that sucked! Is this overfitting!? Let me try that same model with 2000 epochs!

In [ ]:
insurance_model_10 = tf.keras.Sequential([
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(40, activation='relu'),
  tf.keras.layers.Dense(54, activation='relu'),
  tf.keras.layers.Dense(1)  
])

insurance_model_10.compile(loss=tf.keras.losses.mae,
                          optimizer=tf.keras.optimizers.Adam(),
                          metrics=["mae"])

In [ ]:
history10 = insurance_model_10.fit(X_train_normal, y_train, epochs=2000, verbose=0)

In [ ]:
insurance_model_10.evaluate(X_test_normal, y_test)